# Part - 1: Generating h<sup>*</sup>(D) model

## Step - 1: Importing respective libraries and meta-dataset.csv

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 

In [27]:
df = pd.read_csv('./meta-dataset.csv')
df

,Unnamed: 0,beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7,label
0,0,0.116358,0.134907,0.278246,0.470489,0.0,0.0,0.0,0.0,1
1,1,0.096939,0.210459,0.269133,0.423469,0.0,0.0,0.0,0.0,1
2,2,0.067696,0.157957,0.483373,0.290974,0.0,0.0,0.0,0.0,1
3,3,0.085299,0.132486,0.328494,0.453721,0.0,0.0,0.0,0.0,1
4,4,0.026059,0.262215,0.288274,0.423453,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
221,221,0.000000,0.335975,0.664025,0.000000,0.0,0.0,0.0,0.0,1
222,222,0.000000,0.322368,0.677632,0.000000,0.0,0.0,0.0,0.0,1
223,223,0.062405,0.138508,0.512938,0.286149,0.0,0.0,0.0,0.0,1
224,224,0.001678,0.318792,0.677852,0.001678,0.0,0.0,0.0,0.0,1


In [28]:
column_headers = df.columns.values.tolist()
column_headers

for column in column_headers: 
    print(f"{column} = {df[column].corr(df['label'])}")

Unnamed: 0 = 0.034580032509154354
beta_0 = 0.09342538075102236
beta_1 = -0.12233882935886431
beta_2 = -0.17387561852802072
beta_3 = 0.07023988688013583
beta_4 = 0.04900796623224159
beta_5 = 0.0023708454277309174
beta_6 = 0.0054813855785419115
beta_7 = 0.1342352819275331
label = 1.0


In [29]:
# features = df.iloc[:, [2,3,8]]
features = df.iloc[:, 1:-1]
labels = df.iloc[:, [-1]]
scaler = StandardScaler()

In [30]:
features

,beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7
0,0.116358,0.134907,0.278246,0.470489,0.0,0.0,0.0,0.0
1,0.096939,0.210459,0.269133,0.423469,0.0,0.0,0.0,0.0
2,0.067696,0.157957,0.483373,0.290974,0.0,0.0,0.0,0.0
3,0.085299,0.132486,0.328494,0.453721,0.0,0.0,0.0,0.0
4,0.026059,0.262215,0.288274,0.423453,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
221,0.000000,0.335975,0.664025,0.000000,0.0,0.0,0.0,0.0
222,0.000000,0.322368,0.677632,0.000000,0.0,0.0,0.0,0.0
223,0.062405,0.138508,0.512938,0.286149,0.0,0.0,0.0,0.0
224,0.001678,0.318792,0.677852,0.001678,0.0,0.0,0.0,0.0


In [31]:
features_std = scaler.fit_transform(features)
features_std

array([[ 1.21829768, -0.3165277 , -0.12547885, ..., -0.31748475,
        -0.14503491, -0.21312473],
       [ 0.83158988,  0.37134376, -0.16543095, ..., -0.31748475,
        -0.14503491, -0.21312473],
       [ 0.24924362, -0.10666632,  0.7737579 , ..., -0.31748475,
        -0.14503491, -0.21312473],
       ...,
       [ 0.14387595, -0.28374088,  0.90336384, ..., -0.31748475,
        -0.14503491, -0.21312473],
       [-1.06545201,  1.35767216,  1.62631896, ..., -0.31748475,
        -0.14503491, -0.21312473],
       [-1.09886505,  1.59307727,  1.52768379, ..., -0.31748475,
        -0.14503491, -0.21312473]])

In [32]:
labels 

,label
0,1
1,1
2,1
3,1
4,1
...,...
221,1
222,1
223,1
224,1


In [8]:
lable_count = labels.value_counts()
lable_count

label
1        148
2         58
0         20
dtype: int64

In [33]:
label_weights_list = []
label_weights = {
0: len(labels) / (20 * 3),
1: len(labels) / (148 * 3),
2: len(labels) / (58 * 3)
}
label_weights_list.append(label_weights)

In [34]:
label_weights = {
0: len(labels) / (20),
1: len(labels) / (148),
2: len(labels) / (58)
}
label_weights_list.append(label_weights)

In [11]:
label_weights_list

[{0: 3.7666666666666666, 1: 0.509009009009009, 2: 1.2988505747126438},
 {0: 11.3, 1: 1.527027027027027, 2: 3.896551724137931}]

## Step-2: Splitting the dataset to training data and testing data respectively

Using the StandardScalar() function, we normalized the range of the feature values from [0, 1] to [-1, 1]. This is to handle the comparison of very small feature values (i.e 0.00094), thereby improving the performance of our model.

In [35]:
features_train, features_test, labels_train, labels_test = train_test_split(features_std, labels, test_size = 0.20)
print(f'features_train = \n{features_train}')
print(f'features_test = \n{features_test}')
print(f'labels_train = \n{labels_train}')
print(f'labels_test = \n{labels_test}')

features_train = 
[[ 1.13614231 -1.2118509   0.33529539 ... -0.31748475 -0.14503491
  -0.21312473]
 [-0.05732913 -0.24958907 -0.72161925 ... -0.31748475 -0.14503491
  -0.21312473]
 [ 0.49322429 -0.11288774  0.03366064 ... -0.31748475 -0.14503491
  -0.21312473]
 ...
 [ 1.35277102  0.30364706 -1.26004317 ... -0.31748475 -0.14503491
  -0.21312473]
 [-0.12641105 -0.50096513 -0.36797493 ... -0.31748475 -0.14503491
  -0.21312473]
 [-1.09886505  1.92425208  0.70265408 ... -0.31748475 -0.14503491
  -0.21312473]]
features_test = 
[[ 0.41600025 -1.54480666 -1.34525764 -1.26108279 -0.36773141 -0.31748475
   0.26821499  4.91313152]
 [ 0.041259   -0.80346374 -0.59789036  1.54202934 -0.36773141 -0.31748475
  -0.14503491 -0.21312473]
 [ 0.02702423 -0.38662207 -0.7875997   1.53475108 -0.36773141 -0.31748475
  -0.14503491 -0.21312473]
 [ 0.17083014  0.04391859  0.27292764  0.3319796  -0.36773141 -0.31748475
  -0.14503491 -0.21312473]
 [-0.64299858  0.14448387 -0.50018563  1.16855471 -0.36773141 -0.3174

## Step-3: Oversampling Miniority classes and Undersampling Majority Classes

In [36]:
# oversample = SMOTE()
# undersample = RandomUnderSampler()
# steps = [("o", oversample), ("u", undersample)]
# pipeline = Pipeline(steps=steps)
# # Transform the dataset
# features_train, labels_train = pipeline.fit_resample(features_train, labels_train)

## Step-4: Instantiating K-Fold Cross-Validation Algorithm

We used the k-fold cross-validation algorithm to split the dataset into 10 batches. We take 9 batches as training sets and use the remaining batch as a validation set. By doing this process iteratively, we ensure that every sample point is considered in both the training and validation sets which improves the accuracy and performance of the model.

In [37]:
# RepeatedStratifiedKFold_inner = KFold(n_splits=10)
# RepeatedStratifiedKFold_outer = KFold(n_splits=10)
# RepeatedStratifiedKFold_inner = KFold(n_splits=10,shuffle=True)
# RepeatedStratifiedKFold_outer = KFold(n_splits=5,shuffle=True)

#5,
KFold_param = KFold(n_splits=10, shuffle=True)
# RepeatedStratifiedKFold = KFold(n_splits=5, shuffle=True)

## Step-4: Performing Traditional Grid Search on both the Decision Tree model and the K-Nearest-Neighbour model
We perform grid first search on the decision tree model and the K-nearest-neighbour model seperately, as we first find the most optimal variation of each of the models seperately with its respective parameters and then the compare both. The best model is then chosen as our h<sup>*</sup>(D)

### Using f1-score for Accuracy

The accuracy of the models would be compared with f1-score as our dataset is extremely imbalanced with majority of the samples belonging in the class label "1".

The f1-score is well suited to deal with imbalanced datasets because it is derived from the precision and recall scores of a model. Precision measures how many of the “positive” predictions made by the model were correct. Recall measures how many of the positive class samples present in the dataset were correctly identified by the model. As such, f1-score elaborates on a model's classwise performance instead of its overall performance as compared to the usual accuracy metric and is used as our metric in determining h<sup>*</sup>(D)

### Step-4.1: Initializing Decision-Tree model and selecting most relavant hyperparameters

The following hyperparameters were choosen to be used in the for the decision tree model:
1. Criterion: These 3 criterion best represent the information gain at each point of the tree and would thus have the greatest impact on the accuracy of the model.
1. Splitter: We chose the split to be either random or the best to test out the variance. 
1. Max-Depth: The depth will detemine how well the model behaves on unseen data.
1. Min-Samples-Split: Although most of the time it was 2-10, we decided to put in the range of 2 to 10. The reduction in range was done to allow the model to run faster.
1. Class-weight: We have choosen the value of the class-weight parameter to be None because we assume there is no noise in the dataset. The reason for this assumption is because the it in mentioned in the project proposal that the data-preprocessing stages for the pipeline have already been conducted for us.

In [15]:
decision_tree = DecisionTreeClassifier()
criterion_values = ['entropy','gini','log_loss']
splitter_values = ['best', 'random']
max_depth_values = range(5,40)
min_samples_split_values = range(2,10)
min_samples_leaf_values = range(1,10)
max_features_values = range(2,8)
decision_tree_parameters = {'criterion':criterion_values, 'splitter':splitter_values, 'max_depth':max_depth_values, 
                            'min_samples_split':min_samples_split_values, 'class_weight':['balanced'], 'min_samples_split': min_samples_split_values,
                            'min_samples_leaf': min_samples_leaf_values, 'max_features':max_features_values} # Need to verify if the class-weight parameter is important because there is no noise in the dataset


### Step-4.2: Performing Grid Search to find the optimal set of hyperparameters for the decision tree model

In [16]:
decision_tree_classifier = GridSearchCV(decision_tree, decision_tree_parameters, cv=KFold_param)
decision_tree_classifier.fit(features_train,labels_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': ['balanced'],
                         'criterion': ['entropy', 'gini', 'log_loss'],
                         'max_depth': range(5, 40), 'max_features': range(2, 8),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']})

In [17]:
decision_tree_classifier.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 31,
 'max_features': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'splitter': 'random'}

In [18]:
labels_predict = decision_tree_classifier.predict(features_test)
print(classification_report(labels_test, labels_predict))
print(accuracy_score(labels_test, labels_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.83      0.69      0.75        35
           2       0.15      0.25      0.19         8

    accuracy                           0.57        46
   macro avg       0.33      0.31      0.31        46
weighted avg       0.66      0.57      0.60        46

0.5652173913043478


### Step-4.3 Initializing K-Nearest-Neighbour model and selecting most relevant hyperparameters

The following hyperparameters were choosen to be used in the for the decision tree model:
1. knn_neighbours_range: We initially started with a large range from 4 to 113 as a small n would result in a model that is too specific and a large range would result in a model that is too generic. However, after running a few tests, we realised the the number of neighbours returned often ranged from 4 to 10. We thus decided to use this range to save computational power. 
1. weights_values: Uniform would mean all neighbours are equally weighted while distance gives the closer neighbours to the sample point greater influence. We decided to use both values to compare the model's performance on imbalanced data. 
1. algorithm_values: We also explored every algorithm value besides brute force for greater variance without compromising computational power. 

In [19]:
knn = KNeighborsClassifier()
# knn_neighbours_range = range(4,113)
knn_neighbours_range = range(4,10)
weights_values = ['uniform', 'distance']
algorithm_values = ['auto', 'ball_tree', 'kd_tree', ]

knn_parameters = {'n_neighbors': knn_neighbours_range, 'weights':weights_values, 'p':[1,2], # p=2 for euclidian
                    'algorithm': algorithm_values, 'metric':['minkowski']}

### Step-4.4: Performing Grid Search to find the optimal set of hyperparameters for the K-Nearest-Neighbour model

In [20]:
knn_classifier = GridSearchCV(knn, knn_parameters, cv=KFold_param)
knn_classifier.fit(features_train, labels_train)

/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                         'metric': ['minkowski'], 'n_neighbors': range(4, 10),
                         'p': [1, 2], 'weights': ['uniform', 'distance']})

In [21]:
labels_predict = knn_classifier.predict(features_test)
print(classification_report(labels_test, labels_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.76      0.89      0.82        35
           2       0.00      0.00      0.00         8

    accuracy                           0.67        46
   macro avg       0.25      0.30      0.27        46
weighted avg       0.58      0.67      0.62        46



/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Based on the weighted-average f1-scores of the decision tree model and the k-nearest-neighbour model.

In [23]:
h_star_d = knn_classifier
h_star_d

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                         'metric': ['minkowski'], 'n_neighbors': range(4, 10),
                         'p': [1, 2], 'weights': ['uniform', 'distance']})

Saving the model on to a pickle file

In [26]:
filename = 'h_star_d.sav'
pickle.dump(knn_classifier, open(filename, 'wb'))